In [66]:
import pandas as pd
import numpy
from textblob import TextBlob
from string import digits

In [144]:
#import
test_tweets = pd.read_csv('test_tweets.csv')
train_tweets = pd.read_csv('train_tweets.csv')

# 10000 obserwacji (train - 7000, test - 3000) Przy większej ilości brakowało mi pamięci 
train_tweets = train_tweets.iloc[:10000, :]

In [145]:
# Usuwa wszystkie znaki oprócz a-z A-Z i spacji
def leave_alphas(x):
    filtered = ''
    for char in x:
        if((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90 or ord(char) == 32)):
            filtered += char
    return filtered

In [146]:
train_tweets['tweet'] = train_tweets['tweet'].apply(leave_alphas)
test_tweets['tweet'] = test_tweets['tweet'].apply(leave_alphas)

In [94]:
train_tweets.head()

,id,label,tweet
0,1,0,user when a father is dysfunctional and is so...
1,2,0,user user thanks for lyft credit i cant use ca...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation


In [140]:
# NOT IMPLEMENTED:
def cleandata(x):
    obj = TextBlob(x)
    new_blob = []
    for word in obj.words:
        if word != TextBlob.Word('user'):
            new_blob.append(word)
    result = " ".join(str(x) for x in new_blob)
    return result

train_tweets['tweet'] = train_tweets['tweet'].apply(cleandata)
test_tweets['tweet'] = test_tweets['tweet'].apply(cleandata)

In [147]:
# zamiana 0 - 1 na pos - neg
def format_pos_neg(x):
    if x == 1:
        return 'neg'
    if x == 0:
        return 'pos'

train_tweets['label'] = train_tweets['label'].apply(format_pos_neg)

# drop id
train_tweets.drop('id', axis=1, inplace=True)

# zamiana kolejnosci kolumn
cols = train_tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
train_tweets = train_tweets[cols]

In [148]:
# podział na zbiór treningowy i testowy
number = int(0.7 * train_tweets['tweet'].count())
train = train_tweets.iloc[:number, :]
test = train_tweets.iloc[number:, :]

In [149]:
train.count()

tweet    7000
label    7000
dtype: int64

In [150]:
test.count()

tweet    3000
label    3000
dtype: int64

In [151]:
# Tablice tuples [('tweet','label'),('tweet','label')]
subset = train[['tweet','label']]
train_tuples = [tuple(x) for x in subset.values]

subset2 = test[['tweet','label']]
test_tuples = [tuple(x) for x in subset2.values]

In [152]:
from textblob.classifiers import NaiveBayesClassifier

In [134]:
train_tuples

[(' user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction   run',
  'pos'),
 ('user user thanks for lyft credit i cant use cause they dont offer wheelchair vans in pdx    disapointed getthanked',
  'pos'),
 ('  bihday your majesty', 'pos'),
 ('model   i love u take with u all the time in ur   ', 'pos'),
 (' factsguide society now    motivation', 'pos'),
 (' huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo  ',
  'pos'),
 (' user camping tomorrow user user user user user user user danny', 'pos'),
 ('the next school year is the year for exams cant think about that  school exams   hate imagine actorslife revolutionschool girl',
  'pos'),
 ('we won love the land allin cavs champions cleveland clevelandcavaliers   ',
  'pos'),
 (' user user welcome here   im   its so gr  ', 'pos'),
 ('  ireland consumer price index mom climbed from previous  to  in may   blog silver gold forex',
  'pos'),
 ('we ar

In [153]:
cl = NaiveBayesClassifier(train_tuples)

In [154]:
cl.accuracy(test_tuples)

0.9443333333333334

In [158]:
cl.show_informative_features(15)

Most Informative Features
       contains(libtard) = True              neg : pos    =    138.2 : 1.0
        contains(racism) = True              neg : pos    =    120.4 : 1.0
      contains(equality) = True              neg : pos    =     84.7 : 1.0
         contains(bigot) = True              neg : pos    =     66.9 : 1.0
           contains(blm) = True              neg : pos    =     66.9 : 1.0
        contains(racist) = True              neg : pos    =     63.9 : 1.0
       contains(liberal) = True              neg : pos    =     58.0 : 1.0
          contains(maga) = True              neg : pos    =     49.1 : 1.0
      contains(hispanic) = True              neg : pos    =     49.1 : 1.0
       contains(african) = True              neg : pos    =     40.1 : 1.0
            contains(mc) = True              neg : pos    =     40.1 : 1.0
       contains(boricua) = True              neg : pos    =     40.1 : 1.0
         contains(tampa) = True              neg : pos    =     40.1 : 1.0

In [159]:
def classifier(x):
    return cl.classify(x)

In [163]:
# Klasyfikacja pierwszych 50 tweetów z test_tweets.csv
classification = test_tweets.iloc[:50, :]
classification['output'] = classification['tweet'].apply(classifier)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [164]:
classification

,id,tweet,output
0,31963,studiolife aislife requires passion dedication...,pos
1,31964,user white supremacists want everyone to see ...,pos
2,31965,safe ways to heal your acne altwaystoheal h...,pos
3,31966,is the hp and the cursed child book up for res...,pos
4,31967,rd bihday to my amazing hilarious nephew eli...,pos
5,31968,choose to be momtips,pos
6,31969,something inside me dies eyes ness smokeyeye...,pos
7,31970,finishedtattooinkedinkloveit thanksaleeee,pos
8,31971,user user user i will never understand why my...,pos
9,31972,delicious food lovelife capetown mannaepicur...,pos
